In [3]:
import requests
from bs4 import BeautifulSoup 
import re
import unicodedata
import pythainlp.util
from pythainlp.tokenize import word_tokenize
from pythainlp.util import find_keyword
from pythainlp.util import rank
#from pythainlp.summarize import extract_keywords
from pythainlp.summarize import summarize
import itertools
import sqlite3
from urllib.parse import urljoin
import time
import heapq
from keybert import KeyBERT
import nltk

In [7]:
class Thai:
    def __init__(self,data:list):
        self.data_value = data
        self.sentence = self.get_sentence()
        self.keyword = self.get_keyword()
        self.summarize = self.get_summarize()
    def make_sentence(self,list_word):
        self.sentence_value = ''
        for i in list_word:
            for i in list_word:
                if pythainlp.util.countthai(i)<10:
                    list_word.remove(i)
        self.sentence_value = ' '.join(list_word)
        return self.sentence_value
    def get_sentence(self):
        self.sentence_result = self.make_sentence(self.data_value)
        return self.sentence_result
    def get_keyword(self):
        self.keyword_result = {}
        self.keyword_value = word_tokenize(self.sentence, engine="newmm")
        self.keyword_dict = find_keyword(self.keyword_value)
        # Iterate over the keys in the dictionary
        for key in self.keyword_dict:
        # Check if the key is text (i.e., not a space or quotation mark)
            if key.isalpha():
            # If the key is text, add it to the new dictionary
                self.keyword_result[key] = self.keyword_dict[key]
        return self.keyword_result
    def get_summarize(self):
        self.summarize_result =[]
        self.summarize_result = summarize(self.sentence,n=5)
        return self.summarize_result


In [8]:
def scrape_tags(url):
  response = requests.get(url)
  soup = BeautifulSoup(response.text, 'html.parser')
  try:
    title_tag = soup.find('title').text
  except:
    title_tag = soup.find('title')
  p_tags = soup.find_all('p')
  p_list =[]
  for p in p_tags:
    if p.string != None:
      p_list.append(unicodedata.normalize("NFKD", p.string))
  if len(p_list) == 0:
    p_list.append('ไม่พบข้อความในเว็บนี้')
  
  p_tag = "".join(p_list)
  thai_nlp = Thai(p_list)
  keyword = thai_nlp.keyword
  keyword = {k: v for k, v in sorted(keyword.items(), key=lambda item: item[1], reverse=True)}
  keyword = dict(itertools.islice(keyword.items(), 5))
  summarize_article = thai_nlp.summarize
  try:
    location = 'จ.'+max(thai_nlp.get_location().keys())
  except:
    location = 'NONE'
  
  if  title_tag == None:
    try:
      title_tag = summarize_article[0]
    except:
      title_tag = 'NONE'
    
  return p_tag, title_tag, keyword,location

In [9]:
#check language function
def check_lang(url:str):
    data_lang = scrape_tags(url)
    percent = pythainlp.util.countthai(data_lang[0])
    if percent >50:
        return print("Thai language")
    else:
        return print("English language")

In [10]:
check_lang('https://www.bbc.com/afrique/media/photogalleries')

English language


In [11]:
check_lang('https://www.thairath.co.th/news/crime/2633816')


Thai language
